In [1]:
import pandas as pd
import sqlite3

# 3. Basic SQL queries

__Connecting to the DB__

In [10]:
db = sqlite3.connect("c2_database.sqlite")

In [11]:
def run_query(query):
    return pd.read_sql_query(query, db)

__Fetching Data: SQL SELECT Queries__

In [12]:
run_query("SELECT tbl_name FROM sqlite_master;")

,tbl_name
0,station
1,status
2,trip
3,weather


In [13]:
run_query("SELECT * FROM trip LIMIT 5;")

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4100,1392,8/29/2013 9:57,South Van Ness at Market,66,8/29/2013 10:20,San Francisco Caltrain 2 (330 Townsend),69,319,Subscriber,94118
1,4133,1965,8/29/2013 10:17,San Francisco Caltrain 2 (330 Townsend),69,8/29/2013 10:50,Powell Street BART,39,316,Customer,
2,4213,931,8/29/2013 11:14,Grant Avenue at Columbus Avenue,73,8/29/2013 11:30,Post at Kearney,47,473,Customer,20002
3,4282,572,8/29/2013 11:54,San Francisco Caltrain (Townsend at 4th),70,8/29/2013 12:04,Townsend at 7th,65,538,Customer,37206
4,4308,1000,8/29/2013 12:04,Redwood City Caltrain Station,22,8/29/2013 12:21,Redwood City Caltrain Station,22,159,Subscriber,94061


In [14]:
run_query("SELECT start_station_name, end_station_name, duration FROM trip LIMIT 5;")

,start_station_name,end_station_name,duration
0,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1392
1,San Francisco Caltrain 2 (330 Townsend),Powell Street BART,1965
2,Grant Avenue at Columbus Avenue,Post at Kearney,931
3,San Francisco Caltrain (Townsend at 4th),Townsend at 7th,572
4,Redwood City Caltrain Station,Redwood City Caltrain Station,1000


In [15]:
query = '''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE start_station_name = 'Paseo de San Antonio' 
LIMIT 5;
'''

run_query(query)

,start_station_name,end_station_name,duration
0,Paseo de San Antonio,Paseo de San Antonio,539
1,Paseo de San Antonio,St James Park,5334
2,Paseo de San Antonio,Paseo de San Antonio,18009
3,Paseo de San Antonio,San Salvador at 1st,226
4,Paseo de San Antonio,San Salvador at 1st,256


In [16]:
query='''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE duration<100 
LIMIT 5
'''
run_query(query)

,start_station_name,end_station_name,duration
0,Market at Sansome,Market at Sansome,75
1,Market at 10th,South Van Ness at Market,94
2,Powell at Post (Union Square),Powell at Post (Union Square),82
3,Harry Bridges Plaza (Ferry Building),Harry Bridges Plaza (Ferry Building),60
4,Townsend at 7th,Townsend at 7th,83


In [17]:
query='''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE (duration<100) AND (start_station_name = 'Market at 10th')
LIMIT 5;
'''
run_query(query)

,start_station_name,end_station_name,duration
0,Market at 10th,South Van Ness at Market,94
1,Market at 10th,Market at 10th,91
2,Market at 10th,Market at 10th,80
3,Market at 10th,Market at 10th,74
4,Market at 10th,Market at 10th,69


In [18]:
query='''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE start_station_name = 'Paseo de San Antonio' 
ORDER BY duration DESC 
LIMIT 5;
'''
run_query(query)

,start_station_name,end_station_name,duration
0,Paseo de San Antonio,MLK Library,167156
1,Paseo de San Antonio,Paseo de San Antonio,70017
2,Paseo de San Antonio,San Salvador at 1st,53153
3,Paseo de San Antonio,Paseo de San Antonio,18009
4,Paseo de San Antonio,Paseo de San Antonio,11519


# 4. Advanced SQL queries

__AGGREGATE (SUM, AVG, MIN, MAX)__

In [19]:
run_query("SELECT AVG(duration) AS 'Average Duration' FROM trip;")

,Average Duration
0,1070.969428


__GROUP BY__

In [20]:
query='''
SELECT subscription_type, AVG(duration) AS 'Average Duration'  FROM trip 
GROUP BY subscription_type;
'''
run_query(query)

,subscription_type,Average Duration
0,Customer,3715.673978
1,Subscriber,580.480241


__JOIN__

In [22]:
query = '''
SELECT station.city AS "City", COUNT(*) AS "Count"
FROM trip 
INNER JOIN station
ON trip.start_station_name = station.name
GROUP BY station.city;
'''
run_query(query)

,City,Count
0,Mountain View,2232
1,Palo Alto,853
2,Redwood City,409
3,San Francisco,74304
4,San Jose,4748


In [27]:
query = '''
SELECT start.city AS "Start city", end.city AS "End city", trip.duration
FROM trip
INNER JOIN station AS start
ON trip.start_station_name = start.name
INNER JOIN station AS end
ON trip.end_station_name = end.name
WHERE start.city != end.city
ORDER BY duration DESC
LIMIT 3;
'''

run_query(query)


,Start city,End city,duration
0,Mountain View,San Francisco,179330
1,San Jose,Palo Alto,79591
2,San Francisco,Palo Alto,19043


__Subqueries__

In [28]:
query = '''
SELECT sub.*
FROM (SELECT * FROM trip
WHERE duration < 160) sub
WHERE start_station_name='Post at Kearney';
'''
run_query(query)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,17227,140,9/10/2013 20:37,Post at Kearney,47,9/10/2013 20:39,Washington at Kearney,46,395,Subscriber,94133
1,29181,115,9/20/2013 17:17,Post at Kearney,47,9/20/2013 17:19,Post at Kearney,47,345,Customer,
2,31420,156,9/23/2013 10:59,Post at Kearney,47,9/23/2013 11:01,Beale at Market,56,292,Subscriber,94115
3,97477,138,11/15/2013 12:54,Post at Kearney,47,11/15/2013 12:56,Market at Sansome,77,273,Subscriber,94107
4,115555,154,12/4/2013 19:00,Post at Kearney,47,12/4/2013 19:02,Washington at Kearney,46,314,Subscriber,94133
5,153585,141,1/14/2014 23:09,Post at Kearney,47,1/14/2014 23:11,Howard at 2nd,63,381,Subscriber,94105
6,188403,159,2/18/2014 18:43,Post at Kearney,47,2/18/2014 18:45,Washington at Kearney,46,631,Subscriber,94133


In [29]:
query = '''
SELECT sub.*
FROM (SELECT * FROM trip
WHERE duration < 160) sub
WHERE start_station_name='Post at Kearney';
'''
run_query(query)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,17227,140,9/10/2013 20:37,Post at Kearney,47,9/10/2013 20:39,Washington at Kearney,46,395,Subscriber,94133
1,29181,115,9/20/2013 17:17,Post at Kearney,47,9/20/2013 17:19,Post at Kearney,47,345,Customer,
2,31420,156,9/23/2013 10:59,Post at Kearney,47,9/23/2013 11:01,Beale at Market,56,292,Subscriber,94115
3,97477,138,11/15/2013 12:54,Post at Kearney,47,11/15/2013 12:56,Market at Sansome,77,273,Subscriber,94107
4,115555,154,12/4/2013 19:00,Post at Kearney,47,12/4/2013 19:02,Washington at Kearney,46,314,Subscriber,94133
5,153585,141,1/14/2014 23:09,Post at Kearney,47,1/14/2014 23:11,Howard at 2nd,63,381,Subscriber,94105
6,188403,159,2/18/2014 18:43,Post at Kearney,47,2/18/2014 18:45,Washington at Kearney,46,631,Subscriber,94133


# 5. Exercise: SQL queries

__1) What was the duration of the longest trip?__

In [47]:
query='''
SELECT duration FROM trip 
ORDER BY duration DESC
LIMIT 3
'''
run_query(query)

,duration
0,722236
1,688899
2,644771


__2) How many trips were ended at the station ‘2nd at South Park ‘?__


In [39]:
query='''
SELECT end_station_name, COUNT(*) AS "Count"
FROM trip 
WHERE end_station_name = '2nd at South Park'
'''
run_query(query)

,end_station_name,Count
0,2nd at South Park,2150


__3) How many trips had a duration between 1000 and 2000?__


In [50]:
query='''
SELECT COUNT(*) AS 'count'
FROM trip 
WHERE duration BETWEEN 1000 AND 2000;
'''
run_query(query)

,count
0,7225


__4) Which bike was used for the most trips?__


In [52]:
query='''
SELECT bike_id, COUNT(*) AS "Number of trips" FROM trip 
GROUP BY bike_id
ORDER BY "Number of trips" DESC
LIMIT 3
'''
run_query(query)

,bike_id,Number of trips
0,392,277
1,518,264
2,395,264


__5) Which are the top five stations for the number of round trips? (round trips start and end at the same station).__

In [46]:
query='''
SELECT start_station_name, COUNT(*) AS "count"
FROM trip 
WHERE start_station_name = end_station_name
ORDER BY count DESC
LIMIT 2
'''
run_query(query)

,start_station_name,count
0,Redwood City Caltrain Station,2951


# 6. Creating and modifying databases

__Creating a table__

In [53]:
db = sqlite3.connect("my_database.db")

In [54]:
query = "CREATE TABLE customer (id INTEGER PRIMARY KEY, first_name TEXT, last_name TEXT, age INTEGER)"

In [58]:
cursor = db.cursor()

In [60]:
cursor.execute(query)

In [68]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
results = cursor.fetchall()
print(results)

[('customer',)]


__Adding rows__

In [62]:
query = "INSERT INTO customer VALUES  (701, 'Mackenzie', 'Fox', 35)"
cursor.execute(query)
db.commit()

In [69]:
cursor.execute("SELECT * FROM customer;")
results = cursor.fetchall()
print(results)

[(701, 'Mackenzie', 'Fox', 35, 'Geneva')]


__Adding columns__

In [65]:
cursor.execute("ALTER TABLE customer ADD COLUMN city TEXT DEFAULT 'Geneva';")
db.commit()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 cursor.execute("ALTER TABLE customer ADD COLUMN city TEXT DEFAULT 'Geneva';")                │
│   2 db.commit()                                                                                  │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OperationalError: duplicate column name: city

In [66]:
cursor.execute("SELECT * FROM customer;")
results = cursor.fetchall()
print(results)

[(701, 'Mackenzie', 'Fox', 35, 'Geneva')]


__Adding data from a pandas DataFrame__

In [70]:
import pandas as pd

df2 = pd.DataFrame(
    [[702, "Emily", "Joy", 49, "Geneva"]],
    columns=["id", "first_name", "last_name", "age", "city"],
)

In [71]:
df2.to_sql(name="customer", con=db, if_exists="append", index=False)

1

In [72]:
cursor.execute("SELECT * FROM customer;")
results = cursor.fetchall()
print(results)

[(701, 'Mackenzie', 'Fox', 35, 'Geneva'), (702, 'Emily', 'Joy', 49, 'Geneva')]


__Updating rows__

In [73]:
update = """
UPDATE customer
SET city='Zurich'
WHERE id=702;
"""
cursor.execute(update)
db.commit()

In [74]:
cursor.execute("SELECT * FROM customer;")
results = cursor.fetchall()
print(results)

[(701, 'Mackenzie', 'Fox', 35, 'Geneva'), (702, 'Emily', 'Joy', 49, 'Zurich')]


__Deleting rows__

In [75]:
delete = """
DELETE FROM customer
WHERE id=702;
"""
cursor.execute(delete)
db.commit()

In [76]:
cursor.execute("SELECT * FROM customer;")
results = cursor.fetchall()
print(results)

[(701, 'Mackenzie', 'Fox', 35, 'Geneva')]


__Creating a database from a CSV file__

In [77]:
db = sqlite3.connect("songs.db")
cursor = db.cursor()

In [78]:
for chunk in pd.read_csv("c2_songs.csv", chunksize=4):
    chunk.to_sql(name="data", con=db, if_exists="append", index=False)
    print(chunk.iloc[0, 2])

Stairway to Heaven
Black Dog
All My Love
Rebel Rebel
Golden Years


In [79]:
pd.read_sql_query("SELECT * FROM data;", db)

,Musician,Genre,Name,Decade,Minutes
0,Led Zeppelin,hard rock,Stairway to Heaven,70,08:02
1,Led Zeppelin,hard rock,Kashmir,70,08:37
2,Led Zeppelin,hard rock,Immigrant Song,70,02:26
3,Led Zeppelin,hard rock,Whole Lotta Love,60,05:33
4,Led Zeppelin,hard rock,Black Dog,70,04:55
5,Led Zeppelin,hard rock,Good Times Bad Times,60,02:43
6,Led Zeppelin,hard rock,Moby Dick,60,04:25
7,Led Zeppelin,hard rock,Ramble On,60,04:35
8,Led Zeppelin,hard rock,All My Love,70,05:53
9,Led Zeppelin,hard rock,The Song Remains the Same,70,05:24


# 7. Exercise: the Consumer Finance Complaints data

In [110]:
import pandas as pd
import sqlite3

### Importing the csv file as a DataFrame

In [111]:
df = pd.read_csv("c2_consumer_complaints.csv")
df.shape

(65499, 12)

In [112]:
df.head()

,complaint_id,product,issue,company,state,zipcode,submitted_via,date_sent_to_company,date_received,company_response_to_consumer,timely_response,consumer_disputed?
0,511074,Mortgage,"Loan modification,collection,foreclosure",U.S. Bancorp,CA,95993,Referral,09/03/2013,08/30/2013,Closed with explanation,Yes,Yes
1,511080,Mortgage,"Loan servicing, payments, escrow account",Wells Fargo & Company,CA,91104,Referral,09/03/2013,08/30/2013,Closed with explanation,Yes,Yes
2,510473,Credit reporting,Incorrect information on credit report,Wells Fargo & Company,NY,11764,Postal mail,09/18/2013,08/30/2013,Closed with explanation,Yes,No
3,510326,Student loan,Repaying your loan,"Navient Solutions, Inc.",MD,21402,Email,08/30/2013,08/30/2013,Closed with explanation,Yes,Yes
4,511067,Debt collection,False statements or representation,Resurgent Capital Services L.P.,GA,30106,Web,08/30/2013,08/30/2013,Closed with explanation,Yes,Yes


### Creating a database

In [113]:
db = sqlite3.connect("complaints.db")

# defining our helper function for running queries
def run_query(query):
    return pd.read_sql_query(query, db)

### Loading the data into the database

In [114]:
df.to_sql(name="complaints", con=db, if_exists="append", index=False)

65499

In [115]:
# checking that all the data was loaded
run_query("SELECT COUNT(*) FROM complaints")

,COUNT(*)
0,261996


### Computing the number of complaints per submission method

In [116]:
query = """
SELECT submitted_via AS 'Method', COUNT(*) AS 'Number'
FROM complaints
GROUP BY submitted_via
ORDER BY COUNT(*) DESC;
"""
run_query(query)

,Method,Number
0,Web,175788
1,Referral,42348
2,Phone,19896
3,Postal mail,19576
4,Fax,4304
5,Email,84


### Adding the total number of complaints

In [120]:
query = """
SELECT submitted_via AS 'Method', COUNT(*) AS 'Number',
   (SELECT COUNT(*) FROM complaints) AS 'Total'
FROM complaints
GROUP BY submitted_via
ORDER BY COUNT(*) DESC;
"""
run_query(query)

,Method,Number,Total
0,Web,175788,261996
1,Referral,42348,261996
2,Phone,19896,261996
3,Postal mail,19576,261996
4,Fax,4304,261996
5,Email,84,261996


### Computing the percentage of complaints of each submission method

In [123]:
query = """

SELECT *, (CAST (q1.Number AS double precision)/CAST (q1.Total AS double precision))*100 AS 'Percent'
FROM (SELECT submitted_via AS 'Method', COUNT(*) AS 'Number',
   (SELECT COUNT(*) FROM complaints) AS 'Total'
FROM complaints
GROUP BY submitted_via
ORDER BY COUNT(*) DESC) q1;
"""
run_query(query)

,Method,Number,Total,Percent
0,Web,175788,261996,67.095681
1,Referral,42348,261996,16.163606
2,Phone,19896,261996,7.594009
3,Postal mail,19576,261996,7.471870
4,Fax,4304,261996,1.642773
5,Email,84,261996,0.032062
